# Solving Inventory Inefficiencies Using Advanced SQL Analytics
**Dataset**: inventory_forecasting.csv

This notebook performs end-to-end inventory analytics using SQL-style logic, data visualization, and machine learning.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Load the dataset
df = pd.read_csv("inventory_forecasting.csv")
df['Date'] = pd.to_datetime(df['Date'])
df.head()

In [ ]:
# Add time features
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
df['Weekday'] = df['Date'].dt.weekday

# Encode categorical features
le = LabelEncoder()
for col in ['Store ID', 'Product ID', 'Category', 'Region', 'Weather Condition', 'Seasonality']:
    df[col] = le.fit_transform(df[col])

df.info()

In [ ]:
# Units Sold Over Time
plt.figure(figsize=(12, 6))
sns.lineplot(data=df.groupby('Date')['Units Sold'].sum().reset_index(), x='Date', y='Units Sold')
plt.title("Daily Units Sold Over Time")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Units Sold by Seasonality
plt.figure(figsize=(10, 5))
sns.boxplot(data=df, x='Seasonality', y='Units Sold')
plt.title("Units Sold by Season")
plt.tight_layout()
plt.show()

In [ ]:
# Correlation heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(df.corr(numeric_only=True), annot=True, cmap='coolwarm')
plt.title("Feature Correlation Heatmap")
plt.tight_layout()
plt.show()

In [ ]:
# Units Sold by Product Category
plt.figure(figsize=(10, 5))
sns.boxplot(data=df, x='Category', y='Units Sold')
plt.title("Units Sold by Product Category")
plt.tight_layout()
plt.show()

# Units Sold by Region
plt.figure(figsize=(10, 5))
sns.boxplot(data=df, x='Region', y='Units Sold')
plt.title("Units Sold by Region")
plt.tight_layout()
plt.show()

In [ ]:
# Use a sample for faster training
sample_df = df.sample(n=20000, random_state=42)

# Features and target
features = ['Store ID', 'Product ID', 'Category', 'Region', 'Inventory Level',
            'Units Ordered', 'Demand Forecast', 'Price', 'Discount',
            'Weather Condition', 'Holiday/Promotion', 'Competitor Pricing',
            'Seasonality', 'Month', 'Day', 'Weekday']
target = 'Units Sold'

X = sample_df[features]
y = sample_df[target]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predict
y_pred = model.predict(X_test)

# Evaluation
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print(f"RMSE: {rmse:.2f}")
print(f"R² Score: {r2:.2f}")

In [ ]:
# Prediction plot
plt.figure(figsize=(10, 6))
sns.scatterplot(x=y_test, y=y_pred, alpha=0.3)
plt.xlabel("Actual Units Sold")
plt.ylabel("Predicted Units Sold")
plt.title("Prediction vs Actual")
plt.tight_layout()
plt.show()

In [ ]:
# Feature importance
importances = pd.Series(model.feature_importances_, index=features).sort_values(ascending=True)
plt.figure(figsize=(10, 6))
importances.plot(kind='barh')
plt.title("Feature Importance")
plt.tight_layout()
plt.show()

## Gradient Boosting Regressor Model
We’ll now apply a Gradient Boosting Regressor to compare performance with the Random Forest model.

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score

# Model
gb_model = GradientBoostingRegressor(n_estimators=100, random_state=42)
gb_model.fit(X_train, y_train)

# Predict
gb_pred = gb_model.predict(X_test)

# Evaluation
gb_rmse = mean_squared_error(y_test, gb_pred, squared=False)
gb_r2 = r2_score(y_test, gb_pred)

print(f"Gradient Boosting RMSE: {gb_rmse:.2f}")
print(f"Gradient Boosting R² Score: {gb_r2:.2f}")

In [ ]:
# Prediction comparison plot
plt.figure(figsize=(10, 6))
sns.scatterplot(x=y_test, y=gb_pred, alpha=0.3, label='Gradient Boosting')
sns.scatterplot(x=y_test, y=y_pred, alpha=0.3, label='Random Forest')
plt.xlabel("Actual Units Sold")
plt.ylabel("Predicted Units Sold")
plt.title("Model Comparison: Prediction vs Actual")
plt.legend()
plt.tight_layout()
plt.show()